
# Lab | Handling Data Imbalance in Classification Models

For this lab and in the next lessons we will use the dataset 'Healthcare For All' building a model to predict who will donate (TargetB) and then - how much they will give (TargetD) (will be used for lab on Friday). You will be using `files_for_lab/categorical.csv, numerical.csv, and target.csv`. You will need to download the data locally.  Remember to add the files to your .gitignore.

### Scenario

You are revisiting the Healthcare for All Case Study. You are provided with this historical data about Donors and how much they donated. Your task is to build a machine learning model that will help the company identify people who are more likely to donate and then try to predict the donation amount.

### Instructions

In this lab, we will take a look at the degree of imbalance in the data and correct it using the techniques we learned in the class.  You should fork and clone this Repo and begin a new Jupyter notebook.

Here are the steps to be followed (building a simple model without balancing the data):


**Everyone is starting with the same cleaned data**

 

**Begin the Modeling here**
- Look critically at the dtypes of numerical and categorical columns and make changes where appropriate.
- Concatenate numerical and categorical back together again for your X dataframe.  Designate the TargetB as y.
  - Split the data into a training set and a test set.
  - Split further into train_num and train_cat.  Also test_num and test_cat.
  - Scale the features either by using MinMax Scaler or a Standard Scaler. (train_num, test_num)
  - Encode the categorical features using One-Hot Encoding or Ordinal Encoding.  (train_cat, test_cat)
      - **fit** only on train data, transform both train and test
      - again re-concatenate train_num and train_cat as X_train as well as test_num and test_cat as X_test
  - Fit a logistic regression (classification) model on the training data.
  - Check the accuracy on the test data.

**Note**: So far we have not balanced the data.

Managing imbalance in the dataset

- Check for the imbalance.
- Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
- Each time fit the model and see how the accuracy of the model has changed.





In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

from sklearn.model_selection import cross_val_score

pd.set_option('display.max_columns', None)

In [2]:
num = pd.read_csv('files_for_lab/numerical.csv')
cat = pd.read_csv('files_for_lab/categorical.csv')
tar = pd.read_csv('files_for_lab/target.csv')

In [3]:
cat.isna().any()

STATE           False
CLUSTER         False
HOMEOWNR        False
GENDER          False
DATASRCE        False
RFA_2R          False
RFA_2A          False
GEOCODE2        False
DOMAIN_A        False
DOMAIN_B        False
ODATEW_YR       False
ODATEW_MM       False
DOB_YR          False
DOB_MM          False
MINRDATE_YR     False
MINRDATE_MM     False
MAXRDATE_YR     False
MAXRDATE_MM     False
LASTDATE_YR     False
LASTDATE_MM     False
FIRSTDATE_YR    False
FIRSTDATE_MM    False
dtype: bool

In [4]:
cat.dtypes

STATE           object
CLUSTER          int64
HOMEOWNR        object
GENDER          object
DATASRCE         int64
RFA_2R          object
RFA_2A          object
GEOCODE2        object
DOMAIN_A        object
DOMAIN_B         int64
ODATEW_YR        int64
ODATEW_MM        int64
DOB_YR           int64
DOB_MM           int64
MINRDATE_YR      int64
MINRDATE_MM      int64
MAXRDATE_YR      int64
MAXRDATE_MM      int64
LASTDATE_YR      int64
LASTDATE_MM      int64
FIRSTDATE_YR     int64
FIRSTDATE_MM     int64
dtype: object

In [5]:
for i in cat.columns:
    print(i,cat[i].unique())

STATE ['IL' 'CA' 'NC' 'FL' 'other' 'IN' 'MI' 'MO' 'TX' 'WA' 'WI' 'GA']
CLUSTER [36 14 43 44 16 40 39 45 35 53 17 51  2 20 27 12 22 13  8 25 15 42 11 28
 18 24 34  5 31 32 46  3 50  7 37 10 38 21  9 29 30  4 41 49 23 33  1 47
 26 48 19  6 52]
HOMEOWNR ['H' 'U']
GENDER ['F' 'M' 'other']
DATASRCE [3 1 2]
RFA_2R ['L']
RFA_2A ['E' 'G' 'F' 'D']
GEOCODE2 ['C' 'A' 'D' 'B']
DOMAIN_A ['T' 'S' 'R' 'U' 'C']
DOMAIN_B [2 1 3 4]
ODATEW_YR [89 94 90 87 86 88 96 92 93 95 91 97 85 84 83]
ODATEW_MM [ 1  4  2  9 10 11 12  6  8  3  5  7]
DOB_YR [37 52  0 28 20 60 32 23 26 27 54 36 16 43 56 14 48 29 10 13 68 53 46 31
 90 30 58 50 64 18 62 57 44 25 51 65 35 34 38 21 24 11 47 80 39 12 72 22
 40 59 15 17 42 19 61 41 49 33 45 63 81 55 67 69 70 66 76 91 74 71 78  1
 79 73 75  2 84 82 77 87 95 96 88 97 86  8  6  4 92  5 93  7 89  9 85 83]
DOB_MM [12  2  1 11  3  9  7  4 10  8  6  5]
MINRDATE_YR [92 93 91 87 94 95 88 96 89 90 97 86 77 80 75 84 82 78 85 83]
MINRDATE_MM [ 8 10 11  7  5  9  2 12  6  1  4  3]
MAXRDATE

In [6]:
(num.isna().any()).unique()

array([False])

In [7]:
for i in num.columns:
    if num[i].nunique()<75:
        print(i,num[i].nunique())

TCODE 55
INCOME 7
WEALTH1 10
LOCALGOV 55
STATEGOV 65
FEDGOV 55
WEALTH2 10
ETH6 21
ETH7 61
ETH8 60
ETH9 61
ETH10 33
ETH11 34
ETH12 43
ETH14 45
ETH16 66
AGE901 74
AGE902 67
AGE903 59
AGE904 66
AGE905 62
AGE906 55
AGE907 64
CHIL2 70
AGEC2 74
AGEC3 57
AGEC4 43
AGEC5 43
AGEC6 63
CHILC1 62
CHILC2 46
CHILC3 62
CHILC4 56
HHN5 69
HHN6 53
MARR2 51
MARR3 60
DW3 70
HV3 14
HV4 14
HHD7 66
HHD8 17
HHD9 64
HHD12 66
ETHC1 52
ETHC4 45
ETHC6 53
RHP3 27
RHP4 25
HUPA7 48
IC7 65
IC8 60
IC9 66
IC10 67
IC11 50
IC12 36
IC13 31
IC14 71
IC17 72
IC18 71
IC19 73
IC20 54
IC21 43
IC22 36
HHAS2 72
TPE2 70
TPE3 72
TPE4 67
TPE5 50
TPE6 27
TPE7 18
TPE9 56
TPE10 67
TPE11 62
TPE12 59
OCC1 70
OCC2 61
OCC3 35
OCC4 60
OCC5 63
OCC6 27
OCC7 40
OCC8 63
OCC9 69
OCC10 61
OCC11 61
OCC12 48
OCC13 44
EIC2 54
EIC3 51
EIC4 73
EIC5 45
EIC6 34
EIC7 43
EIC8 71
EIC9 50
EIC10 44
EIC11 50
EIC12 41
EIC13 56
EIC14 69
EIC15 58
EIC16 51
OEDC1 54
OEDC2 63
OEDC3 54
OEDC4 66
OEDC6 63
OEDC7 25
EC3 63
EC5 69
EC6 36
EC7 61
EC8 71
SEC3 19
SEC4 54
AFC3

In [8]:
col_transfer = ['TCODE','INCOME','WEALTH1','WEALTH2']

In [9]:
cat[col_transfer] = num[col_transfer]

In [10]:
num = num.drop(col_transfer,axis=1)

In [11]:
cat.dtypes

STATE           object
CLUSTER          int64
HOMEOWNR        object
GENDER          object
DATASRCE         int64
RFA_2R          object
RFA_2A          object
GEOCODE2        object
DOMAIN_A        object
DOMAIN_B         int64
ODATEW_YR        int64
ODATEW_MM        int64
DOB_YR           int64
DOB_MM           int64
MINRDATE_YR      int64
MINRDATE_MM      int64
MAXRDATE_YR      int64
MAXRDATE_MM      int64
LASTDATE_YR      int64
LASTDATE_MM      int64
FIRSTDATE_YR     int64
FIRSTDATE_MM     int64
TCODE            int64
INCOME           int64
WEALTH1          int64
WEALTH2          int64
dtype: object

In [12]:
for i in cat.columns:
    print(i,cat[i].unique())

STATE ['IL' 'CA' 'NC' 'FL' 'other' 'IN' 'MI' 'MO' 'TX' 'WA' 'WI' 'GA']
CLUSTER [36 14 43 44 16 40 39 45 35 53 17 51  2 20 27 12 22 13  8 25 15 42 11 28
 18 24 34  5 31 32 46  3 50  7 37 10 38 21  9 29 30  4 41 49 23 33  1 47
 26 48 19  6 52]
HOMEOWNR ['H' 'U']
GENDER ['F' 'M' 'other']
DATASRCE [3 1 2]
RFA_2R ['L']
RFA_2A ['E' 'G' 'F' 'D']
GEOCODE2 ['C' 'A' 'D' 'B']
DOMAIN_A ['T' 'S' 'R' 'U' 'C']
DOMAIN_B [2 1 3 4]
ODATEW_YR [89 94 90 87 86 88 96 92 93 95 91 97 85 84 83]
ODATEW_MM [ 1  4  2  9 10 11 12  6  8  3  5  7]
DOB_YR [37 52  0 28 20 60 32 23 26 27 54 36 16 43 56 14 48 29 10 13 68 53 46 31
 90 30 58 50 64 18 62 57 44 25 51 65 35 34 38 21 24 11 47 80 39 12 72 22
 40 59 15 17 42 19 61 41 49 33 45 63 81 55 67 69 70 66 76 91 74 71 78  1
 79 73 75  2 84 82 77 87 95 96 88 97 86  8  6  4 92  5 93  7 89  9 85 83]
DOB_MM [12  2  1 11  3  9  7  4 10  8  6  5]
MINRDATE_YR [92 93 91 87 94 95 88 96 89 90 97 86 77 80 75 84 82 78 85 83]
MINRDATE_MM [ 8 10 11  7  5  9  2 12  6  1  4  3]
MAXRDATE

In [13]:
ord_cols = ['DOMAIN_B','INCOME','WEALTH1','WEALTH2']

In [14]:
X = pd.concat([num,cat],axis=1)
y = tar['TARGET_B']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [15]:
X_train_num = X_train[num.columns]
X_test_num = X_test[num.columns]
X_train_cat = X_train[cat.columns]
X_test_cat = X_test[cat.columns]

In [16]:
X_train_num 

AGE  HIT  MALEMILI  MALEVET  VIETVETS  WWIIVETS  LOCALGOV  \
52735  80.000000   24         0       39        18        57         2   
13046  54.000000    0         1       25        33        32         7   
47921  61.611649    0         0        0         0         0         0   
10200  61.611649    0         0       28        20        38         7   
69225  61.611649    0         1       33        29        27         3   
...          ...  ...       ...      ...       ...       ...       ...   
56942  83.000000    0         0       47        19        54        10   
58048  78.000000    0         0       17        33        31         5   
72833  55.000000    0         0       36        33        26         7   
75266  74.000000    0         1       37        25        28        13   
78258  61.611649    9         1       32        36        40        17   

       STATEGOV  FEDGOV  POP901  POP902  POP903  POP90C1  POP90C2  POP90C3  \
52735         5       0    1765     513     675        0        0       99   
13046         9       2    4469    1272    1645        0        0       99   
47921         0       0    1405     502     712       99        0        0   
10200         4       3    2094     614     836        0        0       99   
69225         3       2    1758     519     736       99        0        0   
...         ...     ...     ...     ...     ...      ...      ...      ...   
56942        13       1    1395     375     716       99        0        0   
58048         3       1    2068     589     728       99        0        0   
72833        20       1    1027     283     379       28        0       72   
75266         4       3    1614     496     541       99        0        0   
78258         1       2       3       0       2        0        0        0   

       POP90C4  POP90C5  ETH1  ETH2  ETH3  ETH4  ETH5  ETH6  ETH7  ETH8  ETH9  \
52735       47       53    99     1     0     0     0     0     0     0     0   
13046       49       51    85    14     0     0     0     0     0     0     0   
47921       47       53    99     1     0     0     1     0     0     0     0   
10200       50       50    99     0     0     0     0     0     0     0     0   
69225       47       53    96     2     1     1     1     0     1     0     0   
...        ...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
56942       40       60    96     3     0     1     1     0     0     0     1   
58048       50       50    95     1     0     3     6     0     0     1     1   
72833       46       54    88    11     0     0     2     0     0     0     0   
75266       49       51    83     2     0    13     7     0     5     5     2   
78258       67       33    99     0     0     0     0     0     0     0     0   

       ETH10  ETH11  ETH12  ETH13  ETH14  ETH15  ETH16  AGE901  AGE902  \
52735      0      0      0      0      0      0      0      40      48   
13046      0      0      0      0      0      0      0      33      42   
47921      0      0      0      0      0      0      0      64      65   
10200      0      0      0      0      0      0      0      38      49   
69225      0      0      0      1      0      0      0      29      36   
...      ...    ...    ...    ...    ...    ...    ...     ...     ...   
56942      0      0      0      0      1      0      0      40      47   
58048      1      0      0      3      0      0      3      36      42   
72833      0      0      0      0      1      0      1      38      43   
75266      1      1      0      4      0      1      1      40      46   
78258      0      0      0      0      0      0      0      33      33   

       AGE903  AGE904  AGE905  AGE906  AGE907  CHIL1  CHIL2  CHIL3  AGEC1  \
52735      52      42      50      54      21     44     36     20      9   
13046      45      35      45      48      27     35     42     24     13   
47921      65      58      61      63       7     35     30     34      3   
10200      52    

In [17]:
X_test_num

AGE  HIT  MALEMILI  MALEVET  VIETVETS  WWIIVETS  LOCALGOV  \
34187  48.000000   16         0       30        45        29         4   
58769  32.000000    0         0       35        74         7         7   
78007  61.611649    2         0       32        26        27         4   
23738  61.611649    0         0       41        38        29        10   
93323  35.000000    0         0       46        22        47        14   
...          ...  ...       ...      ...       ...       ...       ...   
27446  64.000000    0         0        0         0         0         0   
78461  61.611649    2         0       32        28        37         7   
1443   41.000000   11         1       26        26        50         7   
2493   69.000000    0         0       27        18        49         6   
47329  50.000000    2         0       17        35        10         6   

       STATEGOV  FEDGOV  POP901  POP902  POP903  POP90C1  POP90C2  POP90C3  \
34187        18      11    1113     284     395        0       81       19   
58769         0       7     942     258     285       99        0        0   
78007         5       1    1785     506     794       96        0        4   
23738         5       3    2526     691    1006        0        0       99   
93323         2       0     683     201     265        0       99        0   
...         ...     ...     ...     ...     ...      ...      ...      ...   
27446         0       0    1065     304     391        0       87       13   
78461         4       3    5960    1574    2427        0       79       21   
1443          3       1    1563     486     564       99        0        0   
2493          3       8     831     207     341        0       99        0   
47329         3       4    2249     624     731       99        0        0   

       POP90C4  POP90C5  ETH1  ETH2  ETH3  ETH4  ETH5  ETH6  ETH7  ETH8  ETH9  \
34187       47       53    83     1     0     0    65     0     0     0     0   
58769       51       49    88    10     0     2     1     0     0     0     0   
78007       47       53    94     4     0     1     1     0     0     0     0   
23738       49       51    95     0     2     1     4     0     0     0     0   
93323       48       52    98     1     0     0     3     0     0     0     0   
...        ...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
27446       48       52    90     5     1     2     9     1     0     0     0   
78461       45       55    99     0     0     0     0     0     0     0     0   
1443        49       51    99     0     0     1     0     1     0     0     0   
2493        47       53    98     0     1     0     0     0     0     0     0   
47329       50       50    95     2     0     2     2     0     0     2     0   

       ETH10  ETH11  ETH12  ETH13  ETH14  ETH15  ETH16  AGE901  AGE902  \
34187      0      0      0     62      0      0      3      28      39   
58769      0      0      0      0      0      0      1      29      37   
78007      0      0      0      0      0      0      1      36      41   
23738      0      0      0      3      0      0      1      37      43   
93323      0      0      0      3      0      0      0      33      45   
...      ...    ...    ...    ...    ...    ...    ...     ...     ...   
27446      0      0      0      8      0      0      2      29      38   
78461      0      0      0      0      0      0      0      38      47   
1443       0      0      0      0      0      0      0      43      49   
2493       0      0      0      0      0      0      0      44      57   
47329      0      0      0      1      0      0      0      30      35   

       AGE903  AGE904  AGE905  AGE906  AGE907  CHIL1  CHIL2  CHIL3  AGEC1  \
34187      45      33      43      48      31     41     39     20     17   
58769      38      28      39      41      35     40     42     18     11   
78007      43      36      43      46      21     38     41     21     10   
23738      44    

In [18]:
X_train_cat

STATE  CLUSTER HOMEOWNR GENDER  DATASRCE RFA_2R RFA_2A GEOCODE2  \
52735     NC       46        H      F         3      L      F        D   
13046     NC       47        U      F         1      L      F        D   
47921     FL       29        H      M         3      L      D        C   
10200     MO       53        H      F         2      L      D        D   
69225  other       25        H      F         1      L      E        B   
...      ...      ...      ...    ...       ...    ...    ...      ...   
56942  other       26        U      F         2      L      F        B   
58048     CA       14        H      F         2      L      E        A   
72833     NC       34        H      M         3      L      G        B   
75266     CA       14        U      F         2      L      E        A   
78258  other       16        H      M         3      L      F        A   

      DOMAIN_A  DOMAIN_B  ODATEW_YR  ODATEW_MM  DOB_YR  DOB_MM  MINRDATE_YR  \
52735        R         2         95          1      18       1           94   
13046        R         2         92          1      44       4           92   
47921        C         2         86          1       0       2           90   
10200        R         3         86          1       0       2           92   
69225        C         2         92          1       0       2           92   
...        ...       ...        ...        ...     ...     ...          ...   
56942        C         2         91          1      15       5           91   
58048        S         1         90          1      20       1           90   
72833        T         1         96          1      43       1           95   
75266        S         1         86          1      24       1           86   
78258        S         2         96          1       0       2           95   

       MINRDATE_MM  MAXRDATE_YR  MAXRDATE_MM  LASTDATE_YR  LASTDATE_MM  \
52735           10           94            9           95            9   
13046            2           95           11           96            3   
47921            7           95            9           96            1   
10200           11           91            4           96           10   
69225            3           94           12           95            6   
...            ...          ...          ...          ...          ...   
56942            1           94           12           95           12   
58048            4           93           10           95           12   
72833            9           96            2           96            2   
75266            9           96            1           96            7   
78258            9           95            9           95            9   

       FIRSTDATE_YR  FIRSTDATE_MM  TCODE  INCOME  WEALTH1  WEALTH2  
52735            94             9     28       5        2        9  
13046            92             2      2       2        3        3  
47921            89             9      0       5        9        9  
10200            86            11      0       2        9        9  
69225            92             3      0       3        6        5  
...             ...           ...    ...     ...      ...      ...  
56942            91             1      2       3        9        7  
58048            90             4      0       1        9        9  
72833            95             9      1       7        8        9  
75266            86             9      2       6        9        9  
78258            95             9      2       2        5        9  

[76329 rows x 26 columns]

In [19]:
X_test_cat

STATE  CLUSTER HOMEOWNR GENDER  DATASRCE RFA_2R RFA_2A GEOCODE2  \
34187     TX       36        H      F         3      L      F        D   
58769  other       18        H      F         3      L      E        B   
78007     NC       22        H      F         3      L      F        B   
23738  other       46        H      F         2      L      F        D   
93323     TX       36        H      M         3      L      D        B   
...      ...      ...      ...    ...       ...    ...    ...      ...   
27446     TX       21        U      M         2      L      F        B   
78461     IN       45        U      F         3      L      F        D   
1443      MI        5        H      M         3      L      G        A   
2493   other       48        U      M         2      L      E        D   
47329  other       14        H      F         3      L      F        B   

      DOMAIN_A  DOMAIN_B  ODATEW_YR  ODATEW_MM  DOB_YR  DOB_MM  MINRDATE_YR  \
34187        T         2         94          1      50       1           94   
58769        S         2         95          1      66       1           95   
78007        C         1         91          1       0       2           91   
23738        R         2         96          1       0       2           95   
93323        T         2         88          1      63       1           95   
...        ...       ...        ...        ...     ...     ...          ...   
27446        S         3         95          1      34       1           95   
78461        R         2         86          1       0       2           87   
1443         U         2         94          1      56       8           93   
2493         R         2         86          1      28       8           87   
47329        S         1         91          1      48       1           91   

       MINRDATE_MM  MAXRDATE_YR  MAXRDATE_MM  LASTDATE_YR  LASTDATE_MM  \
34187            1           95            9           95           12   
58769            1           95            6           95           12   
78007           11           95            6           95            6   
23738           12           95           12           95           12   
93323           11           94            2           95           11   
...            ...          ...          ...          ...          ...   
27446            2           95            6           95           10   
78461            5           95            4           96            3   
1443             9           95            6           95            9   
2493            12           93            2           96            2   
47329            1           95           12           95           12   

       FIRSTDATE_YR  FIRSTDATE_MM  TCODE  INCOME  WEALTH1  WEALTH2  
34187            94             1      2       6        3        9  
58769            95             1      0       4        8        9  
78007            91            11      0       6        9        9  
23738            95            12      0       5        9        9  
93323            88             1      0       5        5        4  
...             ...           ...    ...     ...      ...      ...  
27446            95             2      0       7        9        9  
78461            86            10      2       2        2        3  
1443             93             9      1       5        7        7  
2493             87            12      0       5        9        4  
47329            91             1     28       7        9        9  

[19083 rows x 26 columns]

In [20]:
transformer = MinMaxScaler().fit(X_train_num)

X_train_num = pd.DataFrame(transformer.transform(X_train_num), columns=X_train_num.columns)
X_test_num = pd.DataFrame(transformer.transform(X_test_num), columns=X_test_num.columns)

In [21]:
X_test_num

AGE       HIT  MALEMILI   MALEVET  VIETVETS  WWIIVETS  LOCALGOV  \
0      0.484536  0.066390  0.000000  0.303030  0.454545  0.292929  0.040404   
1      0.319588  0.000000  0.000000  0.353535  0.747475  0.070707  0.070707   
2      0.624862  0.008299  0.000000  0.323232  0.262626  0.272727  0.040404   
3      0.624862  0.000000  0.000000  0.414141  0.383838  0.292929  0.101010   
4      0.350515  0.000000  0.000000  0.464646  0.222222  0.474747  0.141414   
...         ...       ...       ...       ...       ...       ...       ...   
19078  0.649485  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
19079  0.624862  0.008299  0.000000  0.323232  0.282828  0.373737  0.070707   
19080  0.412371  0.045643  0.010101  0.262626  0.262626  0.505051  0.070707   
19081  0.701031  0.000000  0.000000  0.272727  0.181818  0.494949  0.060606   
19082  0.505155  0.008299  0.000000  0.171717  0.353535  0.101010  0.060606   

       STATEGOV    FEDGOV    POP901    POP902    POP903   POP90C1   POP90C2  \
0      0.181818  0.126437  0.011276  0.011950  0.011157  0.000000  0.818182   
1      0.000000  0.080460  0.009544  0.010856  0.008050  1.000000  0.000000   
2      0.050505  0.011494  0.018085  0.021291  0.022427  0.969697  0.000000   
3      0.050505  0.034483  0.025592  0.029075  0.028416  0.000000  0.000000   
4      0.020202  0.000000  0.006920  0.008457  0.007485  0.000000  1.000000   
...         ...       ...       ...       ...       ...       ...       ...   
19078  0.000000  0.000000  0.010790  0.012791  0.011044  0.000000  0.878788   
19079  0.040404  0.034483  0.060384  0.066229  0.068554  0.000000  0.797980   
19080  0.030303  0.011494  0.015836  0.020449  0.015931  1.000000  0.000000   
19081  0.030303  0.091954  0.008419  0.008710  0.009632  0.000000  1.000000   
19082  0.030303  0.045977  0.022786  0.026256  0.020648  1.000000  0.000000   

        POP90C3   POP90C4   POP90C5      ETH1      ETH2      ETH3      ETH4  \
0      0.191919  0.474747  0.535354  0.838384  0.010101  0.000000  0.000000   
1      0.000000  0.515152  0.494949  0.888889  0.101010  0.000000  0.020202   
2      0.040404  0.474747  0.535354  0.949495  0.040404  0.000000  0.010101   
3      1.000000  0.494949  0.515152  0.959596  0.000000  0.020408  0.010101   
4      0.000000  0.484848  0.525253  0.989899  0.010101  0.000000  0.000000   
...         ...       ...       ...       ...       ...       ...       ...   
19078  0.131313  0.484848  0.525253  0.909091  0.050505  0.010204  0.020202   
19079  0.212121  0.454545  0.555556  1.000000  0.000000  0.000000  0.000000   
19080  0.000000  0.494949  0.515152  1.000000  0.000000  0.000000  0.010101   
19081  0.000000  0.474747  0.535354  0.989899  0.000000  0.010204  0.000000   
19082  0.000000  0.505051  0.505051  0.959596  0.020202  0.000000  0.020202   

           ETH5      ETH6  ETH7      ETH8  ETH9  ETH10  ETH11  ETH12  \
0      0.656566  0.000000   0.0  0.000000   0.0    0.0    0.0    0.0   
1      0.010101  0.000000   0.0  0.000000   0.0    0.0    0.0    0.0   
2      0.010101  0.000000   0.0  0.000000   0.0    0.0    0.0    0.0   
3      0.040404  0.000000   0.0  0.000000   0.0    0.0    0.0    0.0   
4      0.030303  0.000000   0.0  0.000000   0.0    0.0    0.0    0.0   
...         ...       ...   ...       ...   ...    ...    ...    ...   
19078  0.090909  0.045455   0.0  0.000000   0.0    0.0    0.0    0.0   
19079  0.000000  0.000000   0.0  0.000000   0.0    0.0    0.0    0.0   
19080  0.000000  0.045455   0.0  0.000000   0.0    0.0    0.0    0.0   
19081  0.000000  0.000000   0.0  0.000000   0.0    0.0    0.0    0.0   
19082  0.020202  0.000000   0.0  0.020202   0.0    0.0    0.0    0.0   

          ETH13  ETH14  ETH15     ETH16    AGE901    AGE902    AGE903  \
0      0.639175    0.0    0.0  0.034884  0.333333  0.464286  0.535714   
1      0.000000    0.0    0.0  0.011628  0.345238  0.440476  0.452381   
2      0.000000    0.0    0.0  0.011628  0.428571  0.488095  0.511905   
3      0

In [22]:
X_train_num

AGE       HIT  MALEMILI   MALEVET  VIETVETS  WWIIVETS  LOCALGOV  \
0      0.814433  0.099585  0.000000  0.393939  0.181818  0.575758  0.020202   
1      0.546392  0.000000  0.010101  0.252525  0.333333  0.323232  0.070707   
2      0.624862  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
3      0.624862  0.000000  0.000000  0.282828  0.202020  0.383838  0.070707   
4      0.624862  0.000000  0.010101  0.333333  0.292929  0.272727  0.030303   
...         ...       ...       ...       ...       ...       ...       ...   
76324  0.845361  0.000000  0.000000  0.474747  0.191919  0.545455  0.101010   
76325  0.793814  0.000000  0.000000  0.171717  0.333333  0.313131  0.050505   
76326  0.556701  0.000000  0.000000  0.363636  0.333333  0.262626  0.070707   
76327  0.752577  0.000000  0.010101  0.373737  0.252525  0.282828  0.131313   
76328  0.624862  0.037344  0.010101  0.323232  0.363636  0.404040  0.171717   

       STATEGOV    FEDGOV    POP901    POP902    POP903   POP90C1  POP90C2  \
0      0.050505  0.000000  0.017882  0.021585  0.019066  0.000000      0.0   
1      0.090909  0.022989  0.045278  0.053522  0.046465  0.000000      0.0   
2      0.000000  0.000000  0.014235  0.021123  0.020111  1.000000      0.0   
3      0.040404  0.034483  0.021216  0.025835  0.023614  0.000000      0.0   
4      0.030303  0.022989  0.017811  0.021838  0.020789  1.000000      0.0   
...         ...       ...       ...       ...       ...       ...      ...   
76324  0.131313  0.011494  0.014134  0.015779  0.020224  1.000000      0.0   
76325  0.030303  0.011494  0.020952  0.024783  0.020563  1.000000      0.0   
76326  0.202020  0.011494  0.010405  0.011908  0.010705  0.282828      0.0   
76327  0.040404  0.034483  0.016352  0.020870  0.015281  1.000000      0.0   
76328  0.010101  0.022989  0.000030  0.000000  0.000056  0.000000      0.0   

        POP90C3   POP90C4   POP90C5      ETH1      ETH2      ETH3      ETH4  \
0      1.000000  0.474747  0.535354  1.000000  0.010101  0.000000  0.000000   
1      1.000000  0.494949  0.515152  0.858586  0.141414  0.000000  0.000000   
2      0.000000  0.474747  0.535354  1.000000  0.010101  0.000000  0.000000   
3      1.000000  0.505051  0.505051  1.000000  0.000000  0.000000  0.000000   
4      0.000000  0.474747  0.535354  0.969697  0.020202  0.010204  0.010101   
...         ...       ...       ...       ...       ...       ...       ...   
76324  0.000000  0.404040  0.606061  0.969697  0.030303  0.000000  0.010101   
76325  0.000000  0.505051  0.505051  0.959596  0.010101  0.000000  0.030303   
76326  0.727273  0.464646  0.545455  0.888889  0.111111  0.000000  0.000000   
76327  0.000000  0.494949  0.515152  0.838384  0.020202  0.000000  0.131313   
76328  0.000000  0.676768  0.333333  1.000000  0.000000  0.000000  0.000000   

           ETH5  ETH6      ETH7      ETH8      ETH9     ETH10     ETH11  \
0      0.000000   0.0  0.000000  0.000000  0.000000  0.000000  0.000000   
1      0.000000   0.0  0.000000  0.000000  0.000000  0.000000  0.000000   
2      0.010101   0.0  0.000000  0.000000  0.000000  0.000000  0.000000   
3      0.000000   0.0  0.000000  0.000000  0.000000  0.000000  0.000000   
4      0.010101   0.0  0.014925  0.000000  0.000000  0.000000  0.000000   
...         ...   ...       ...       ...       ...       ...       ...   
76324  0.010101   0.0  0.000000  0.000000  0.014925  0.000000  0.000000   
76325  0.060606   0.0  0.000000  0.010101  0.014925  0.021739  0.000000   
76326  0.020202   0.0  0.000000  0.000000  0.000000  0.000000  0.000000   
76327  0.070707   0.0  0.074627  0.050505  0.029851  0.021739  0.021277   
76328  0.000000   0.0  0.000000  0.000000  0.000000  0.000000  0.000000   

       ETH12     ETH13     ETH14     ETH15     ETH16    AGE901    AGE902  \
0        0.0  0.000000  0.000000  0.000000  0.000000  0.476190  0.571429   
1        0.0  0.000000  0.000000  0.000000  0.000000  0.392857  0.500000   
2        0.0  0.000000  0.000000  0.000000  0.00

In [23]:
X_train_cat_ord = X_train_cat[ord_cols]
X_test_cat_ord = X_test_cat[ord_cols]

In [24]:
ordinal = OrdinalEncoder().fit(X_train_cat_ord)

X_train_cat_ord = pd.DataFrame(ordinal.transform(X_train_cat_ord), columns=X_train_cat_ord.columns)
X_test_cat_ord = pd.DataFrame(ordinal.transform(X_test_cat_ord), columns=X_test_cat_ord.columns)

In [25]:
X_train_cat_ord

DOMAIN_B  INCOME  WEALTH1  WEALTH2
0           1.0     4.0      2.0      9.0
1           1.0     1.0      3.0      3.0
2           1.0     4.0      9.0      9.0
3           2.0     1.0      9.0      9.0
4           1.0     2.0      6.0      5.0
...         ...     ...      ...      ...
76324       1.0     2.0      9.0      7.0
76325       0.0     0.0      9.0      9.0
76326       0.0     6.0      8.0      9.0
76327       0.0     5.0      9.0      9.0
76328       1.0     1.0      5.0      9.0

[76329 rows x 4 columns]

In [26]:
X_test_cat_ord

DOMAIN_B  INCOME  WEALTH1  WEALTH2
0           1.0     5.0      3.0      9.0
1           1.0     3.0      8.0      9.0
2           0.0     5.0      9.0      9.0
3           1.0     4.0      9.0      9.0
4           1.0     4.0      5.0      4.0
...         ...     ...      ...      ...
19078       2.0     6.0      9.0      9.0
19079       1.0     1.0      2.0      3.0
19080       1.0     4.0      7.0      7.0
19081       1.0     4.0      9.0      4.0
19082       0.0     6.0      9.0      9.0

[19083 rows x 4 columns]

In [27]:
X_train_cat_onehot = X_train_cat.drop(columns=ord_cols)
X_test_cat_onehot = X_test_cat.drop(columns=ord_cols)

In [28]:
X_train_cat_onehot.columns == X_test_cat_onehot.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [29]:
X_train_cat_onehot.dtypes

STATE           object
CLUSTER          int64
HOMEOWNR        object
GENDER          object
DATASRCE         int64
RFA_2R          object
RFA_2A          object
GEOCODE2        object
DOMAIN_A        object
ODATEW_YR        int64
ODATEW_MM        int64
DOB_YR           int64
DOB_MM           int64
MINRDATE_YR      int64
MINRDATE_MM      int64
MAXRDATE_YR      int64
MAXRDATE_MM      int64
LASTDATE_YR      int64
LASTDATE_MM      int64
FIRSTDATE_YR     int64
FIRSTDATE_MM     int64
TCODE            int64
dtype: object

In [30]:
onehot = OneHotEncoder(drop='first',handle_unknown='ignore').fit(X_train_cat_onehot)

X_train_cat_onehot_encoded = onehot.transform(X_train_cat_onehot)
X_test_cat_onehot_encoded = onehot.transform(X_test_cat_onehot)

/Users/iremnisa.kilinc/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:241: UserWarning: Found unknown categories in columns [11, 19, 21] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [31]:
X_train_cat_onehot = pd.DataFrame(X_train_cat_onehot_encoded.toarray(), columns=onehot.get_feature_names_out(X_train_cat_onehot.columns))
X_test_cat_onehot = pd.DataFrame(X_test_cat_onehot_encoded.toarray(), columns=onehot.get_feature_names_out(X_test_cat_onehot.columns))

In [32]:
(X_test_cat_ord.isnull().any()).unique()

array([False])

In [33]:
X_train = pd.concat([X_train_num,X_train_cat_ord,X_train_cat_onehot],axis=1)
X_test = pd.concat([X_test_num,X_test_cat_ord,X_test_cat_onehot],axis=1)

In [34]:
(X_test.isnull().any()).value_counts()

False    677
Name: count, dtype: int64

In [35]:
from sklearn.linear_model import LogisticRegression

classification = LogisticRegression(random_state=7, solver='saga',
                  multi_class='multinomial', max_iter=100).fit(X_train, y_train)

classification.score(X_test, y_test)

/Users/iremnisa.kilinc/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


0.9474401299586019

In [42]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [43]:
pred = classification.predict(X_test)

In [44]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[18080,     0],
       [ 1003,     0]])

In [45]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=127)
X_train_SMOTE,y_train_SMOTE = sm.fit_resample(X_train,y_train)
X_train_SMOTE.shape

(144978, 677)

In [46]:
LR = LogisticRegression(random_state=7, solver='saga',
                  multi_class='multinomial')
LR.fit(X_train_SMOTE, y_train_SMOTE)
pred2 = LR.predict(X_test)

print("accuracy: ", LR.score(X_test, y_test))
print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

accuracy:  0.6214431693130011
precision:  0.0
recall:  0.0
f1:  0.0


/Users/iremnisa.kilinc/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/iremnisa.kilinc/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1497: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [47]:
confusion_matrix(y_test,pred2)

array([[11362,  6718],
       [  506,   497]])

# Lab | Random Forests

For this lab, you will be using the .CSV files provided in the files_for_lab folder. These are cleaned versions of the learningSet data from the Case Study 'Healthcare for All'.
You may continue in the Jupyter Notebook you created yesterday. There is no need to fork and clone this Repo.

### Instructions

- Apply the Random Forests algorithm AFTER upscaling the data to deal with the imbalance.
- Use Feature Selections that you have learned in class to decide if you want to use all of the features (Variance Threshold, RFE, PCA, etc.)
- Re-run the Random Forest algorithm to determine if the Feature Selection has improved the results.
- Discuss the output and its impact in the business scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the business?

In [48]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=10, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             max_samples=0.8, # fraction of X-train to use in each tree
                             random_state=7)

clf.fit(X_train_SMOTE, y_train_SMOTE)

RandomForestClassifier(max_depth=10, max_samples=0.8, min_samples_leaf=20,
                       min_samples_split=20, random_state=7)

In [49]:
clf.score(X_train_SMOTE, y_train_SMOTE)

0.9578004938680351

In [50]:
clf.score(X_test, y_test)

0.918618665828224

In [54]:
pred3 = clf.predict(X_test)

print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

precision:  0.07822085889570553
recall:  0.05084745762711865
f1:  0.06163141993957704


In [55]:
confusion_matrix(y_test,pred3)

array([[17479,   601],
       [  952,    51]])

In [57]:
from sklearn.decomposition import PCA
pca = PCA(0.9).fit(X_train_SMOTE)
pca.explained_variance_ratio_

array([0.30411188, 0.16448487, 0.06768406, 0.03656555, 0.0221289 ,
       0.014751  , 0.01387058, 0.01240639, 0.01163122, 0.01029087,
       0.00975744, 0.00896683, 0.00797068, 0.0077314 , 0.00722463,
       0.00715134, 0.00658084, 0.00626459, 0.00611314, 0.00591341,
       0.00573722, 0.00550073, 0.00543544, 0.00532033, 0.00517472,
       0.00506153, 0.00488472, 0.00482276, 0.00475773, 0.00452512,
       0.00431353, 0.00426392, 0.00417205, 0.00399719, 0.00376594,
       0.0037058 , 0.00340498, 0.00323762, 0.00312458, 0.00307507,
       0.00302533, 0.00297358, 0.00292871, 0.00290961, 0.00285075,
       0.00273845, 0.00261572, 0.00252788, 0.00249277, 0.00247826,
       0.00236537, 0.00233202, 0.00227153, 0.00221311, 0.00215314,
       0.00205372, 0.00199869, 0.00193868, 0.00185954, 0.00178197,
       0.00177118, 0.00168626, 0.00165866, 0.00163601, 0.00162889,
       0.00160853, 0.00157468, 0.0015428 , 0.00150413, 0.0014556 ,
       0.00142833, 0.00141571, 0.00138477, 0.00137908, 0.00134

In [60]:
X_train_pca = pca.transform(X_train_SMOTE)
X_test_pca = pca.transform(X_test)

In [61]:
clf.fit(X_train_pca, y_train_SMOTE)

RandomForestClassifier(max_depth=10, max_samples=0.8, min_samples_leaf=20,
                       min_samples_split=20, random_state=7)

In [62]:
clf.score(X_train_pca, y_train_SMOTE)

0.8139096966436287

In [63]:
clf.score(X_test_pca, y_test)

0.6888853953780852

In [67]:
pred4 = clf.predict(X_test_pca)

print("precision: ",precision_score(y_test,pred4))
print("recall: ",recall_score(y_test,pred4))
print("f1: ",f1_score(y_test,pred4))

precision:  0.06960921144452198
recall:  0.39780658025922233
f1:  0.11848552338530066


In [68]:
confusion_matrix(y_test,pred4)

array([[12747,  5333],
       [  604,   399]])